In [1]:
import pandas as pd

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1️⃣ Load the CSV fil
df = pd.read_csv("/mnt/c/Users/abhis/Downloads/archive (6)/Data/train.csv")
# Create image_path column only if file exists
# df['image_path'] = df['image_id'].apply(
#     lambda x: os.path.join(folder_path, x) if os.path.exists(os.path.join(folder_path, x)) else None
# )

# 2️⃣ Add full image paths
df['image_path'] = df['image_id'].apply(lambda x: os.path.join("/mnt/c/Users/abhis/Downloads/archive (6)/Data/train_images", x))
# Make sure label column is string
df['label'] = df['label'].astype(str)

# Then split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])



df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/abhis/Downloads/archive (6)/Data/train.csv'

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,            # normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),    # resize all images
    class_mode='categorical',  # or 'binary' if only 2 classes
    batch_size=32
)

In [ ]:
val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

In [ ]:
print("Class indices:", train_gen.class_indices)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Get number of classes
num_classes = len(train_gen.class_indices)


model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
# Load the trained model
model = load_model("best_model.h5")

# Load a new image for testing
img_path = "/mnt/c/Users/abhis/Downloads/archive (6)/Data/train_images/987636835.jpg"   # path to your test image
plt.imshow(image.load_img(img_path))
plt.axis('off')
plt.title("Image going for prediction")
plt.show()
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

# Predict
pred = model.predict(img_array)
class_idx = np.argmax(pred, axis=1)[0]

print("Predicted class index:", class_idx)


In [ ]:
pip install matplotlib

In [ ]:
!pip install streamlit 

In [ ]:
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from PIL import Image

# --- Load your trained model (.keras or .h5) ---
model = tf.keras.models.load_model("best_model.h5")

# --- Define class labels ---
class_labels = ['class_0', 'class_1', 'class_2', 'class_3']  # change to your actual class names

# --- Page title ---
st.title("🌿 Plant Disease Prediction App")
st.write("Upload an image of a leaf to predict its disease category.")

# --- File uploader ---
uploaded_file = st.file_uploader("Choose a leaf image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)

    # Preprocess image for prediction
    img = img.resize((224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    pred = model.predict(img_array)
    predicted_class = np.argmax(pred, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    confidence = round(100 * np.max(pred), 2)

    # Show result
    st.subheader(f"🌱 Predicted Disease: **{predicted_label}**")
    st.write(f"🧠 Confidence: **{confidence}%**")
